## Bibliotecas usadas 

In [1]:
import pandas as pd
import requests
import json 
from unicodedata import normalize

## Pegando dados do governo

In [2]:
url = 'http://coronavirus.pi.gov.br/'
pagina = pd.read_html(url)
dados_governo = pagina[0]

In [3]:
dados_governo.columns = ['Cidade', 'Confirmados', 'Óbitos', 'Letalidade'] 

## Pegando dados do IBGE

In [4]:
jsonCi = json.loads(requests.get('https://servicodados.ibge.gov.br/api/v1/localidades/estados/22/municipios').text)

## Gerando o nome e IBGE ID de todas as cidades do Piauí

In [5]:
nomes = []
ids = []
for i in jsonCi:
    nomes.append(i['nome'])
    ids.append(i['id'])

## Funções para limpeza dos dados do governo

In [6]:
def remover_acentos(txt):
    return normalize('NFKD', txt).encode('ASCII', 'ignore').decode('ASCII')

In [7]:
def numero_letras_erradas_menor_dois(cidade1, cidade2):
    diferenca = 0
    if len(cidade1) != len(cidade2):
        return False
    for i, j in list(zip(list(cidade1), list(cidade2))):
        if i != j:
            diferenca += 1
    return diferenca < 2

In [8]:
def compara_cidades(cidade1, cidade2):
    if cidade1.lower() == cidade2.lower():
        return True
    elif remover_acentos(cidade1.lower()) == remover_acentos(cidade2.lower()):
        return True
    elif numero_letras_erradas_menor_dois(remover_acentos(cidade1.lower()), remover_acentos(cidade2.lower())):
        return True
    else: 
        return False

## Gerar o numero de casos e óbitos para cada cidade

In [9]:
casos = [0 for i in range(len(nomes))]
obitos = [0 for i in range(len(nomes))]
for itrIB, cIBGE in enumerate(nomes):
    for itrCO, cCOVID in enumerate(dados_governo.Cidade):
        if compara_cidades(cIBGE, cCOVID):
            casos[itrIB] = dados_governo.Confirmados.iloc[itrCO]
            obitos[itrIB] = dados_governo['Óbitos'].iloc[itrCO]
            

## Gerando DataFrame

In [10]:
dados_piaui = pd.DataFrame({'name': nomes, 
                            'id': ids,
                            'casos': casos,
                            'obitos': obitos})
dados_piaui

,name,id,casos,obitos
0,Acauã,2200053,0,0
1,Agricolândia,2200103,0,0
2,Água Branca,2200202,13,0
3,Alagoinha do Piauí,2200251,0,0
4,Alegrete do Piauí,2200277,0,0
...,...,...,...,...
219,Várzea Branca,2211357,0,0
220,Várzea Grande,2211407,1,0
221,Vera Mendes,2211506,0,0
222,Vila Nova do Piauí,2211605,2,0


## Exportando CSV

In [11]:
dados_piaui.to_csv('dados_piaui.csv', index=False, header=False)